<a href="https://colab.research.google.com/github/sgroark/genAI/blob/main/HW4part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras import layers
import time
from IPython import display

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# download dataset
!kaggle datasets download -d jhoward/lsun_bedroom

Dataset URL: https://www.kaggle.com/datasets/jhoward/lsun_bedroom
License(s): ODbL-1.0
100% 8.89G/8.89G [00:47<00:00, 82.7MB/s]
100% 8.89G/8.89G [00:47<00:00, 199MB/s] 


In [ ]:
# create folder to store unzipped images
images_folder = '/content/unzipped_images'
os.makedirs(images_folder, exist_ok=True)

In [ ]:
import zipfile

zip_file_path = '/content/lsun_bedroom.zip'
extract_to = '/content/unzipped_images'

# Extract the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# Verify the extracted contents
os.listdir(extract_to)

# List the contents of the 'content' directory
os.listdir('/content/unzipped_images')

['sample', 'data0']

In [ ]:
from PIL import Image

# loads + processes a single image

def load_image(file_path):
    image = Image.open(file_path)
    image = image.resize((64, 64))
    image = np.array(image) / 127.5 - 1
    return image

# generator function

def image_generator(image_paths):
    for path in image_paths:
        yield load_image(path)

root_folder = '/content/unzipped_images/data0/lsun/bedroom'

# create list of paths
image_paths = []

for root, dirs, files in os.walk(root_folder):
    for file in files:
        if file.endswith('.jpg'):
            image_paths.append(os.path.join(root, file))

# take subset of only 5000 images

subset_images = image_paths[:5000]

train_dataset = tf.data.Dataset.from_generator(
    lambda: image_generator(subset_images),
    output_signature=tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32)
)

BUFFER_SIZE = len(subset_images)
BATCH_SIZE = 512

train_dataset = train_dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# store array of subsetted training images
train_images = np.array([load_image(path) for path in subset_images])